In [1]:
import acgan
import twodlearn as tdl
import tensorflow as tf
import tensorflow_datasets as tfds
from functools import reduce
#tf.enable_eager_execution()

BATCH_SIZE = 32
session = tf.InteractiveSession()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
#tfds.list_builders()

In [3]:
dataset, info = tfds.load('mnist', with_info=True,
                          split=tfds.Split.TRAIN)
dataset = dataset.shuffle(1024).repeat()\
                 .map(lambda x: (tf.cast(x['image'], tf.float32)/255.0))\
                 .batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
input_shape = dataset.output_shapes.as_list()

Instructions for updating:
Colocations handled automatically by placer.


W0310 06:01:25.955709 139795072358208 deprecation.py:323] From /data/marinodl/research/2019/AssociativeGAN/env/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model = acgan.model.DCGAN(
    embedding_size=100,
    generator={'init_shape':(7, 7, 256),
               'units': [128, 64, 1],
               'kernels': 5,
               'strides': [1, 2, 2]},
    discriminator={'units':[64, 128],
                   'kernels':5,
                   'strides':2}
)

In [5]:
iter = dataset.make_one_shot_iterator()
xreal = iter.get_next()

In [6]:
gen = model.generator_trainer(BATCH_SIZE)
#dis = model.discriminator_trainer(BATCH_SIZE, input_shape[1:])
dis = model.discriminator_trainer(BATCH_SIZE, xreal=xreal)

Instructions for updating:
Use tf.cast instead.


W0310 06:01:27.902876 139795072358208 deprecation.py:323] From /data/marinodl/research/2019/AssociativeGAN/env/lib/python3.6/site-packages/tensorflow/python/keras/utils/losses_utils.py:170: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


W0310 06:01:27.927157 139795072358208 deprecation.py:323] From /data/marinodl/research/2019/AssociativeGAN/env/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [7]:
tdl.core.variables_initializer(model.generator.variables).run()
tdl.core.variables_initializer(model.discriminator.variables).run()
tf.global_variables_initializer().run()

In [12]:
#optimizer = tf.train.AdamOptimizer()
session.run(dis.step)